### prepare_seed_json

This notebook prepares a formatted .json file with for one state with census ids etc filled in.
The JSON is formatted in a way that later allows an easy import into our database to display it values on a map.

In [60]:
import pandas as pd # package for high-performance, easy-to-use data structures and data analysis
import numpy as np # fundamental package for scientific computing with Python

In [61]:
from subprocess import check_output
print(check_output(["ls", "./export/census_tracts_lookup"]).decode("utf8"))

Alabama.csv
Alaska.csv
Arizona.csv
Arkansas.csv
California.csv
Colorado.csv
Connecticut.csv
Delaware.csv
District of Columbia.csv
Florida.csv
Georgia.csv
Hawaii.csv
Idaho.csv
Illinois.csv
Indiana.csv
Iowa.csv
Kansas.csv
Kentucky.csv
Louisiana.csv
Maine.csv
Maryland.csv
Massachusetts.csv
Michigan.csv
Minnesota.csv
Mississippi.csv
Missouri.csv
Montana.csv
Nebraska.csv
Nevada.csv
New Hampshire.csv
New Jersey.csv
New Mexico.csv
New York.csv
North Carolina.csv
North Dakota.csv
Ohio.csv
Oklahoma.csv
Oregon.csv
Pennsylvania.csv
Rhode Island.csv
South Carolina.csv
South Dakota.csv
Tennessee.csv
Texas.csv
Utah.csv
Vermont.csv
Virginia.csv
Washington.csv
West Virginia.csv
Wisconsin.csv
Wyoming.csv
all_states.csv
faulty_census_tracts.csv



In [62]:
census_lookup = pd.read_csv("./export/census_tracts_lookup/all_states.csv", delimiter=",")

In [63]:
census_lookup.head()

,state_code,state,county_code,county,census_tracts,census_tract_number
0,1,Alabama,1,Autauga County,1517,207.0
1,1,Alabama,1,Autauga County,1201,210.0
2,1,Alabama,1,Autauga County,1836,201.0
3,1,Alabama,1,Autauga County,1311,211.0
4,1,Alabama,1,Autauga County,2805,204.0


In [64]:
import os
import simplejson

def make_sure_directory_exists(path):
    if not os.path.exists(path):
        os.makedirs(path)
        
def export_to_json(path, filename, dic):
    make_sure_directory_exists(path)
    filepath = os.path.join(path, filename)
    with open(filepath, "w") as f:
        f.write(simplejson.dumps(dic, indent=4, sort_keys=False))


In [85]:
import json

def parse_state_geojson():
    ''' Returns a dict where each key corresponds to one state_code 
        and its value to the coordinates from the geojson.'''
    
    filepath = './input_data/geojson/us_states.geojson'
    
    states_geojson = open(filepath).read()
    states_geojson = str(states_geojson).strip('\n')
    states_geojson = str(states_geojson).strip(';')
    
    json_data = json.loads(states_geojson)
    
    return_dic = {}
    for feature in json_data['features']:
        return_dic[feature['id']] = feature['geometry']['coordinates']
    
    return return_dic
    
def parse_county_geojson():
    ''' Returns a dict where each key corresponds to one a string
    of '{:02d}-{:03d}'.format(state_code, county_code) and its value 
    to the coordinates from the geojson for the county'''
    
    filepath = './input_data/geojson/us_counties.geojson'
    
    counties_geojson = open(filepath).read()
    counties_geojson = str(counties_geojson).strip('\n')
    counties_geojson = str(counties_geojson).strip(';')
    
    json_data = json.loads(counties_geojson)
    
    return_dic = {}
    for feature in json_data['features']:
        key = feature['properties']['STATEFP'] + '-' + feature['properties']['COUNTYFP']
        return_dic[key] = feature['geometry']['coordinates']
    
    return return_dic

def parse_census_geojson():
    ''' Returns a dict where each key corresponds to one census_tract_number 
    and its value to the coordinates from the geojson.'''
    
    # TODO: Implement this
    return {}

In [94]:
'''Generates a JSON File with the required map data to display data interactively on the map.'''

state_geojson_lookup = parse_state_geojson()
county_geojson_lookup = parse_county_geojson()
census_geojson_lookup = parse_census_geojson()

def get_geojson_dict(state_id=None, county_id=None, census_tract_number=None, kind=None):
    
    coordinates = [ ['//TODO' ] ]
    
    if kind is not None:
        if kind == 'state'and state_id is not None:
            if state_geojson_lookup.get('{:02d}'.format(state_id)) is not None:
                coordinates = state_geojson_lookup.get('{:02d}'.format(state_id))
        if kind == 'county'and state_id is not None and county_id is not None:
            # TODO: Make sure the ids are formatted properly
            if county_geojson_lookup.get('{:02d}-{:03d}'.format(state_id, county_id)) is not None:
                coordinates = county_geojson_lookup.get('{:02d}-{:03d}'.format(state_id, county_id))
        if kind == 'census' and state_id is not None and county_id is not None and census_id is not None:
            # TODO: Make sure the ids are formatted properly
            if census_geojson_lookup.get(id) is not None:
                coordinates = state_geojson_lookup.get(id)

    return {
        'type': 'Feature',
        'geometry': {
            'type': 'Polygon',
             'coordinates': coordinates
        }
    }

def get_census_dict(census_tract, census_tract_number, count_code, state_code):
    return {
        'census_tract': census_tract,
        'census_tract_number': census_tract_number,
        'geojson': get_geojson_dict()
    }

def get_county_dict(df, county, county_code, state_code):
    filtered_df = df.loc[df['county_code'] == county_code]
    # TODO: figure out how to deal with faulty census tracts
    filtered_df = filtered_df.dropna(axis=0, how='any', subset=['census_tract_number'])
    census_tracts = filtered_df[['census_tracts', 'census_tract_number']].drop_duplicates()
    return {
        'county': county,
        'county_code': county_code,
        'geojson': get_geojson_dict(state_id=state_code, county_id=county_code, kind='county'),
        'census_tracts': [get_census_dict(ct, ctn, county_code, state_code) for (ct, ctn) in census_tracts.values ]
    }

def get_state_dict(df, state, state_code):
    filtered_df = df.loc[df['state_code'] == state_code]
    counties = filtered_df[['county', 'county_code']].drop_duplicates()
    return {
        'state': state,
        'state_code': state_code,
        'geojson': get_geojson_dict(state_id=state_code, kind='state'),
        'counties': [get_county_dict(filtered_df, c, cc, state_code) for (c, cc) in counties.values ]
    }

   
    
verbose = True
path = './export/json'

states = census_lookup[['state', 'state_code']].drop_duplicates()
count = 0
for (state, state_code) in states.values:
    count += 1
    export_dict = get_state_dict(census_lookup, state, state_code)

    filename = '2016_{}_HDMA.json'.format(state)
    filepath = os.path.join(path, filename)
    verbose and print("Saving :: {:02d} of {} to '{}'".format(count, len(states.values), filepath))
    export_to_json(path, filename, export_dict)        

    
print('Finished')

         

Saving :: 01 of 51 to './export/json/2016_Alabama_HDMA.json'
Saving :: 02 of 51 to './export/json/2016_Alaska_HDMA.json'
Saving :: 03 of 51 to './export/json/2016_Arizona_HDMA.json'
Saving :: 04 of 51 to './export/json/2016_Arkansas_HDMA.json'
Saving :: 05 of 51 to './export/json/2016_California_HDMA.json'
Saving :: 06 of 51 to './export/json/2016_Colorado_HDMA.json'
Saving :: 07 of 51 to './export/json/2016_Connecticut_HDMA.json'
Saving :: 08 of 51 to './export/json/2016_Delaware_HDMA.json'
Saving :: 09 of 51 to './export/json/2016_District of Columbia_HDMA.json'
Saving :: 10 of 51 to './export/json/2016_Florida_HDMA.json'
Saving :: 11 of 51 to './export/json/2016_Georgia_HDMA.json'
Saving :: 12 of 51 to './export/json/2016_Hawaii_HDMA.json'
Saving :: 13 of 51 to './export/json/2016_Idaho_HDMA.json'
Saving :: 14 of 51 to './export/json/2016_Illinois_HDMA.json'
Saving :: 15 of 51 to './export/json/2016_Indiana_HDMA.json'
Saving :: 16 of 51 to './export/json/2016_Iowa_HDMA.json'
Saving 